# Clean Data

## Do the usefuls imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection  import train_test_split

## Add mapping for type of kitchen + state of building + type of property, change nan in 0

In [2]:
df = pd.read_csv('60k.csv')

labels = df.columns
print(labels)

for label in labels :

    df[label] =df[label].replace(-1, np.nan)
    
map_type_of_property = {
                        "HOUSE" : 0,
                        "APARTMENT" : 1
                        }

map_state_of_the_building = {
                            "AS_NEW" : 1.0,
                            "JUST_RENOVATED" : 1.0,
                            "GOOD" : 1.0,
                            "TO_BE_DONE_UP" : 0.0,
                            "TO_RENOVATE" : 0.0,
                            "TO_RESTORE" : 0.0,
                            "NO_INFO" : 0.0
                            }

map_kitchen_type = {            
                            "USA_HYPER_EQUIPPED" : 1.0,
                            "HYPER_EQUIPPED" : 1.0,
                            "USA_INSTALLED" : 1.0,
                            "INSTALLED" : 1.0,
                            "SEMI_EQUIPPED" : 1.0,
                            "USA_SEMI_EQUIPPED" : 1.0,
                            "USA_UNINSTALLED" : 0.0,
                            "NOT_INSTALLED" : 0.0,
                            "NO_INFO" : 0.0
                            }

df["type_of_property"] = df["type_of_property"].map(map_type_of_property)
df["state_of_the_building"] = df["state_of_the_building"].map(map_state_of_the_building)
df["kitchen_type"] = df["kitchen_type"].map(map_kitchen_type)
df["furnished"] = df["furnished"].fillna(0.0)
df["swimming_pool"] = df["swimming_pool"].fillna(0.0)
df["garden"] = df["garden"].fillna(0.0)
df["garden_surface"] = df["garden_surface"].fillna(0.0)
df["land_surface"] = df["land_surface"].fillna(0.0)
df["number_of_facades"] = df["number_of_facades"].fillna(0.0)
df["open_fire"] = df["open_fire"].fillna(0.0)
df["terrace"] = df["terrace"].fillna(0.0)
df["terrace_surface"] = df["terrace_surface"].fillna(0.0)

df = df[(df["type_of_sale"] == "residential_sale")]
df = df[(df["type_of_property"] == 0)|(df["type_of_property"] == 1)]
df["priceperm2"] = df["price"]/df["surface"]
df = df[~(df["priceperm2"].isna())]
df = df[~(df["number_of_bedrooms"].isna())]

Index(['Unnamed: 0', 'id', 'locality', 'postal_code', 'region', 'province',
       'type_of_property', 'subtype_of_property', 'type_of_sale', 'price',
       'number_of_bedrooms', 'surface', 'kitchen_type',
       'fully_equipped_kitchen', 'furnished', 'open_fire', 'terrace',
       'terrace_surface', 'garden', 'garden_surface', 'land_surface',
       'number_of_facades', 'swimming_pool', 'state_of_the_building'],
      dtype='object')


## Load Belgium district mapping

In [3]:
column_names_zip = ['country_code','postal_code','municipality','region','region_code','province', 'province_code','arrondissement', 'nbr_mun', 'longitude', 'latitude','other']
zip_code = pd.read_csv('be_zipcode.txt', sep="\t", header=None , names=column_names_zip).drop_duplicates(['postal_code'])
zip_code = zip_code[['postal_code','arrondissement']]

## Add district column in Immoweb dataframe

In [4]:
dict_postalcode_district = {}

for index , row in zip_code.iterrows() :
    dict_postalcode_district[row["postal_code"]] = row["arrondissement"]

df["district"] = df["postal_code"].map(dict_postalcode_district)

## remove useless column and create training and testing dataset

In [5]:
df = df.drop("Unnamed: 0", axis = 1)
df = df.drop("subtype_of_property", axis = 1)
df = df.drop("fully_equipped_kitchen", axis = 1)
df = df.drop("type_of_sale", axis = 1)
df = df.drop("locality", axis = 1)

df = shuffle(df)

df_train, df_dev_test = train_test_split(df,test_size = 0.3)
df_dev, df_test = train_test_split(df_dev_test,test_size = 0.5)

df_train = df_train.reset_index(drop = True)
df_dev = df_dev.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

df_train.to_csv("df_train.csv",index = False)
df_dev.to_csv("df_dev.csv",index = False)
df_test.to_csv("df_test.csv",index = False)

"""
check that all nan are removed

df.shape

labels = df.columns

for label in labels :

     df = df[df[label].notna()]

df.shape
"""

'\ncheck that all nan are removed\n\ndf.shape\n\nlabels = df.columns\n\nfor label in labels :\n\n     df = df[df[label].notna()]\n\ndf.shape\n'